# Sentiment Analysis on IMDb with Soft Prompting [PEFT + T5]

This project fine-tunes a T5-small model on the IMDb dataset using Soft Prompting from the [PEFT library](https://github.com/huggingface/peft).  
The goal is to perform sentiment classification [positive/negative] while training only a small set of soft prompt parameters, making it efficient and lightweight.


# Imports and Device Setup

In [3]:

!pip install torch peft transformers

  Using cached torch-2.9.1-cp313-cp313-manylinux_2_28_x86_64.whl.metadata (30 kB)
  Using cached transformers-4.57.3-py3-none-any.whl.metadata (43 kB)
  Using cached triton-3.5.1-cp313-cp313-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (1.7 kB)
  Using cached accelerate-1.12.0-py3-none-any.whl.metadata (19 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
Using cached torch-2.9.1-cp313-cp313-manylinux_2_28_x86_64.whl (899.7 MB)
Using cached triton-3.5.1-cp313-cp313-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (170.5 MB)
Using cached transformers-4.57.3-py3-none-any.whl (12.0 MB)
Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.3 MB)
Using cached accelerate-1.12.0-py3-none-any.whl (380 kB)
  Attempting uninstall: triton
    Found existing installation: triton 3.5.0
    Uninstalling triton-3.5.0:
      Successfully uninstalled triton-3.5.0
  Attempting uninstall: tokeniz

In [4]:
import subprocess
import sys

result = subprocess.run([sys.executable, '-m', 'pip', 'list'], 
                       capture_output=True, text=True)

print("ML/NLP Related Packages:\n")
keywords = ['transformers', 'torch', 'tensorflow', 'datasets', 'tokenizers', 
            'adapters', 'accelerate', 'evaluate', 'sentencepiece', 'sacremoses']

for line in result.stdout.split('\n'):
    if any(keyword in line.lower() for keyword in keywords):
        print(line)

ML/NLP Related Packages:

accelerate                1.12.0
adapters                  1.2.0
evaluate                  0.4.6
tokenizers                0.22.1
torch                     2.9.1
transformers              4.57.3


In [7]:
pip install datasets

  Using cached datasets-4.4.1-py3-none-any.whl.metadata (19 kB)
Using cached datasets-4.4.1-py3-none-any.whl (511 kB)
Note: you may need to restart the kernel to use updated packages.


In [8]:
# General
import torch

# Dataset
import datasets
from datasets import load_dataset

# Transformers
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq,
)

# PEFT
from peft import get_peft_model, PromptEncoderConfig

# Evaluation
import evaluate


In [9]:
#Use GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


# Load Pre-trained Model

In [10]:
#load the pre-trained t5 model
model = AutoModelForSeq2SeqLM.from_pretrained(
    "google-t5/t5-small",
    torch_dtype=torch.float16,
    device_map="auto"
    )

`torch_dtype` is deprecated! Use `dtype` instead!
/home/khushi/miniconda3/lib/python3.13/site-packages/accelerate/utils/modeling.py:1598: UserWarning: The following device_map keys do not match any submodules in the model: ['decoder.embed_tokens', 'encoder.embed_tokens']
  warnings.warn(


# Load IMDb Dataset

In [11]:
# load the IMDB splits: train, test and unsupervised (we do not use unsupervised)
dataset = load_dataset("imdb", split=['train', 'test', 'unsupervised'])

# wrap into a DatasetDict for convenience
dataset = datasets.DatasetDict({ "train" : dataset[0], "test" : dataset[1], "unsupervised" : dataset[2] })

# split the original training set into train and validation
dataset["train"], dataset["validation"] = dataset["train"].train_test_split(test_size=0.2, seed=42).values()

# Tokenization

In [12]:
# load T5-small tokenizer
tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-small")

# if pad token is missing, use eos token as pad
if tokenizer.pad_token_id is None:
  tokenizer.pad_token_id = tokenizer.eos_token_id



In [13]:
# tokenize a batch of IMDB examples for T5.
def tokenize(batch):

  # tokenize input texts
  model_inputs = tokenizer(batch["text"], padding=False, truncation=True)

  # convert numeric labels to text labels
  label = ["negative" if label == 0 else "positive" for label in batch["label"]]

  # tokenize labels as targets
  labels = tokenizer(text_target=label, truncation=True)
  model_inputs["labels"] = labels["input_ids"]

  return model_inputs

In [14]:
# apply tokenization to all splits
tokenized_ds = dataset.map(
    tokenize,
    batched=True,
    num_proc=4,
    remove_columns=dataset["train"].column_names,
)

In [15]:
train_ds = tokenized_ds["train"] # train dataset
eval_ds = tokenized_ds["validation"]
test_ds = tokenized_ds["test"]

# Prompt Encoder Setup

In [16]:
num_virtual_tokens = 10

In [17]:
#configure prompt encoder
prompt_config = PromptEncoderConfig(task_type="SEQ_2_SEQ_LM", num_virtual_tokens=num_virtual_tokens, encoder_hidden_size=64)#,intially num_virtual_tokens=20,encoder_hidden_size=128
prompt_model = get_peft_model(model, prompt_config) #integrate model with prompt configuration
prompt_model.print_trainable_parameters() #print parameter statistics

trainable params: 80,512 || all params: 60,587,136 || trainable%: 0.1329


In [18]:
# Data collator and metric
data_collator = DataCollatorForSeq2Seq(tokenizer, model=prompt_model)
from evaluate import load

accuracy = load("accuracy")


# compute accuracy from generated predictions and labels
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # Replace -100 so labels can be decoded
    labels = [[token if token != -100 else tokenizer.pad_token_id for token in label]
              for label in labels ]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    pred_labels = [1 if pred.strip() == "positive" else 0 for pred in decoded_preds]
    true_labels = [1 if lab.strip() == "positive" else 0 for lab in decoded_labels]
    return accuracy.compute(predictions=pred_labels, references=true_labels)



In [24]:
import numpy as np

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # Sometimes preds is a tuple (logits, ...) or (generated_ids, ...)
    if isinstance(preds, tuple):
        preds = preds[0]

    # Convert to numpy arrays explicitly
    preds = np.asarray(preds)
    labels = np.asarray(labels)

    # Replace -100 with pad token so we can decode labels
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    # Now safely decode
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Map text -> 0/1
    pred_labels = [1 if pred.strip() == "positive" else 0 for pred in decoded_preds]
    true_labels = [1 if lab.strip() == "positive" else 0 for lab in decoded_labels]

    return accuracy.compute(predictions=pred_labels, references=true_labels)


In [ ]:
batch_size=128
training_args = Seq2SeqTrainingArguments(
      output_dir="./soft_prompt_imdb_results",
      learning_rate=5e-3,
      per_device_train_batch_size=batch_size,
      gradient_accumulation_steps=4, #accumulate gradients over 4 steps, effective batch size = batch_size * 4
      gradient_checkpointing_kwargs={"use_reentrant": False},
      per_device_eval_batch_size=batch_size,
      fp16=True, #half precision training
      num_train_epochs=3,
      logging_strategy="epoch",
      label_names=["labels"],
      gradient_checkpointing=True, #new #saves memory by recomputing gradients during back prop
      save_strategy="epoch", #checkpoints
      eval_strategy="epoch",
      # save_steps=50,
      load_best_model_at_end = True,
      predict_with_generate=True,
        
    )

    
# Initialize Trainer
prompt_trainer = Seq2SeqTrainer(
    model=prompt_model, # Pass the PEFT model to the trainer
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/tmp/ipykernel_104874/322965294.py:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  prompt_trainer = Seq2SeqTrainer(


#Train the Model

In [ ]:
prompt_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.185400,0.150023,0.871200
2,0.165300,0.144134,0.879000
3,0.160100,0.147442,0.878000


#Evaluation

In [ ]:
results = prompt_trainer.evaluate()
print(results)

{'eval_loss': 0.15266896784305573, 'eval_accuracy': 0.87236, 'eval_runtime': 410.4523, 'eval_samples_per_second': 60.908, 'eval_steps_per_second': 7.614, 'epoch': 3.0}


In [ ]:
sample = dataset["test"][0]
inputs = tokenizer(f"review: {sample['text']} sentiment:", return_tensors="pt").to("cuda")
outputs = prompt_model.generate(**inputs, max_new_tokens=3)
print("Prediction:", tokenizer.decode(outputs[0], skip_special_tokens=True))
print("Ground truth:", "positive" if sample['label']==1 else "negative")


Prediction: negative
Ground truth: negative


#Sample Predictions

In [ ]:
samples = eval_ds.select(range(5))

inputs = {
    "input_ids": torch.tensor(samples["input_ids"]).to(prompt_model.device),
    "attention_mask": torch.tensor(samples["attention_mask"]).to(prompt_model.device),
}

with torch.no_grad():
    outputs = prompt_model.generate(**inputs, max_new_tokens=3)

decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
decoded_labels = tokenizer.batch_decode(samples["labels"], skip_special_tokens=True)

print("Predictions:", decoded_preds)
print("Labels:", decoded_labels)


Predictions: ['negative', 'negative', 'negative', 'negative', 'positive']
Labels: ['negative', 'negative', 'negative', 'negative', 'negative']


#Trainable Parameters

In [ ]:
total = sum(p.numel() for p in prompt_model.parameters())
trainable = sum(p.numel() for p in prompt_model.parameters() if p.requires_grad)
pct = 100.0 * trainable / total if total > 0 else 0.0
print(f"Trainable: {trainable:,} / Total: {total:,} ({pct:.6f}%)")

Trainable: 121,472 / Total: 60,628,096 (0.200356%)


#Conclusion

The model achieves ~87% accuracy on IMDB sentiment classification while training only 0.2% of parameters.

Only ~121K parameters were updated, compared to >60M if full fine-tuning was used. The model achieves competitive results which makes prompt-tuning an excellent alternative to full fine-tuning in low-resource or deployment-constrained settings.